# *Gromacs_py* basic example

Here is an example of a short simulation of the SH3 domain of phospholipase Cγ1.
Seven successive steps are used:

1. Topologie creation using ``create_top.py``.
2. Minimisation of the structure using ``minimize_pdb.py``.
3. Solvation of the system using ``solvate_ions.py``.
4. Minimisation of the system using ``minimize_pdb.py``.
5. Equilibration of the system using ``equi_3_step.py``.
6. Production run using ``production.py``.
7. Extension of the production run using ``extend.py``.

## Import

In [1]:
import sys
import os
import urllib.request  
sys.path.insert(0, os.path.abspath('..'))
import gromacs_py.gromacs.gmx5 as gmx

Gromacs version is 5.1.4
FORCEFIELD_PATH = /Users/smurail/Documents/Code/gromacs_py/gromacs_py/gromacs/template


## Simulation setup

In [2]:
# Folder location
DATA_OUT = 'data_sim'
PDB_ID = '1Y0M'

# Setup
ion_C = 0.15

# Energy Minimisation
em_folder = os.path.join(DATA_OUT, 'em')
em_step_number = 1000

# Equilibration
dt = 0.005

## Get PDB file from the ``rcsb.org`` website 

In [6]:
os.makedirs(DATA_OUT, exist_ok = True)

raw_pdb = urllib.request.urlretrieve('http://files.rcsb.org/download/{}.pdb'.format(PDB_ID),
                           '{}/{}.pdb'.format(DATA_OUT, PDB_ID))

In [8]:
raw_pdb[0]

'data_sim/1Y0M.pdb'

## Define the simulation system

In [12]:
md_sys = gmx.GmxSys(name=PDB_ID, coor_file=raw_pdb[0])
md_sys.display()


name :	 1Y0M
coor_file :	 data_sim/1Y0M.pdb
nt :	 0
ntmpi :	 0


## Create topologie:

In [13]:
md_sys.prepare_top(out_folder=os.path.join(DATA_OUT, 'prot_top'), vsite='hydrogens')
md_sys.create_box(dist=1.0, box_type="dodecahedron", check_file_out=True)
md_sys.display()


Succeed to read file ../1Y0M.pdb ,  648 atoms found
Succeed to save file tmp_pdb2pqr.pdb
pdb2pqr.py --ff CHARMM --ffout CHARMM --chain tmp_pdb2pqr.pdb 00_1Y0M.pqr
Succeed to read file 00_1Y0M.pqr ,  996 atoms found
Chain: A  Residue: 0 to 60
Succeed to save file 01_1Y0M_good_his.pdb
-Create topologie
gmx pdb2gmx -f 01_1Y0M_good_his.pdb -o 1Y0M_pdb2gmx.pdb -p 1Y0M_pdb2gmx.top -i 1Y0M_posre.itp -water tip3p -ff charmm36-jul2017 -ignh -vsite hydrogens
Molecule topologie present in 1Y0M_pdb2gmx.top , extract the topologie in a separate file: 1Y0M_pdb2gmx.itp
Protein_chain_A
-ITP file: 1Y0M_pdb2gmx.itp
-molecules defined in the itp file:
* Protein_chain_A
Rewrite topologie: 1Y0M_pdb2gmx.top
-Create pbc box
gmx editconf -f data_sim/prot_top/1Y0M_pdb2gmx.pdb -o data_sim/prot_top/1Y0M_pdb2gmx_box.pdb -bt dodecahedron -d 1.0
name :	 1Y0M
coor_file :	 data_sim/prot_top/1Y0M_pdb2gmx_box.pdb
top_file :	 data_sim/prot_top/1Y0M_pdb2gmx.top
nt :	 0
ntmpi :	 0


In [14]:
md_sys.em_2_steps(out_folder=DATA_OUT+'/prot_em',
                  no_constr_nsteps=em_step_number,
                  constr_nsteps=em_step_number,
                  posres="",
                  create_box_flag=False)


-Create the tpr file  Init_em_1Y0M.tpr
gmx grompp -f Init_em_1Y0M.mdp -c ../prot_top/1Y0M_pdb2gmx_box.pdb -r ../prot_top/1Y0M_pdb2gmx_box.pdb -p ../prot_top/1Y0M_pdb2gmx.top -po out_Init_em_1Y0M.mdp -o Init_em_1Y0M.tpr -maxwarn 1
-Launch the simulation Init_em_1Y0M.tpr
gmx mdrun -s Init_em_1Y0M.tpr -deffnm Init_em_1Y0M -nt 0 -ntmpi 0 -nsteps -2 -nocopyright
-Create the tpr file  1Y0M.tpr
gmx grompp -f 1Y0M.mdp -c Init_em_1Y0M.gro -r Init_em_1Y0M.gro -p ../prot_top/1Y0M_pdb2gmx.top -po out_1Y0M.mdp -o 1Y0M.tpr -maxwarn 1
-Launch the simulation 1Y0M.tpr
gmx mdrun -s 1Y0M.tpr -deffnm 1Y0M -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [ ]:
os.path.join(DATA_OUT, 'em')

In [ ]:
del gmx
import gromacs_py.gromacs.gmx5 as gmx